# MSCI 332 - Project Part 2

> **Team Members:** Maan Patel, Dhruv Hari, Nishesh Jagga, Edward Jeong <br>
> **Course:** MSCI 332 <br>
> **Topic:** Optimal EV chargers in a network <br>


# Problem Definition

The problem definition is finding the optimal placements and number of Electric Vehicle chargers in a network. This part of the project focuses on solving the problem with heuristics rather than a mathematical model. 



## Construction Heuristic

A simple and "not so smart" solution to a problem that makes several decisions to solve the problem from cognition. 

A construction heuristic is a type of heuristic that loops a basic solution until the final results are obtained. 

## Meta-Heuristic

A meta-heuristic is a type of heuristic that improves the basic heuristic by working on a current feasbile solution. The heurisitc then tries to do swaps or improvements in the network to see if the optimal value of the function is increased or decreased depending on the objective function. 

## Problem Approach

We will first create a construction heuristic that will solve the problem. 
Using the basic feasible solution obtained, a meta-heuristic will be used to improve this solution

# Creating multiple networks

To ensure the heuristics performs well and consistently, we will create multiple networks such that feasible solutions can be generated in differnt network or a real-world network of roads

In [ ]:
# Code taken from MSCI 332 Tutorial 2
# and enhanced into a class to allow testing across mutl
from itertools import combinations, groupby
import networkx as nx
import random
import matplotlib.pyplot as plt
import numpy as np


RANGE_OF_CAR = 336
MIN_ARC_LENGTH = 80
MAX_ARC_LENGTH = 200
from networkx import exception

# original version from https://stackoverflow.com/a/61961881
def gnp_random_connected_graph(n, p):
    """
    Generates a random undirected graph, similarly to an Erdős-Rényi 
    graph, but enforcing that the resulting graph is conneted
    """
    edges = combinations(range(n), 2)
    G = nx.Graph()
    for i in range(n):
      G.add_node(i)

    if p <= 0:
        return G
    if p >= 1:
        return nx.complete_graph(n, create_using=G)
    for _, node_edges in groupby(edges, key=lambda x: x[0]):
      node_edges = list(node_edges)
      index = np.random.randint(len(node_edges))
      random_edge = node_edges[index]
      G.add_edge(random_edge[0], random_edge[1], weight=np.random.randint(MIN_ARC_LENGTH, MAX_ARC_LENGTH))
      for e in node_edges:
          if np.random.random() < p:
              G.add_edge(*e, weight=np.random.randint(MIN_ARC_LENGTH, MAX_ARC_LENGTH))
    return G
  
def create_network(random, nodes, prob_of_connection):
  np.random.seed(random)
  probability = prob_of_connection
  G = gnp_random_connected_graph(nodes, probability)
  labels = nx.get_edge_attributes(G,'weight')
  pos = nx.spring_layout(G,k=5)
  plt.figure(figsize=(11,9))
  nx.draw_networkx_edge_labels(G,pos, edge_labels=labels)
  nx.draw(G, pos, with_labels=True, node_color='lightblue',node_size=1000)
  plt.show()
  return {"graph":G, "pos":pos}

In [ ]:
from networkx import exception

def display_solution(graph_with_pos,chargers,cost):
  G = graph_with_pos["graph"]
  pos = graph_with_pos["pos"]
  # output
  try:
    print(f"\nCost from Heuristic: ${cost}")
    color_map = []
    optimal_intersections = []
    for node, count in chargers.items():
      print(f"{int(count)} chargers will be place at intersection {node}")

    for node in G:
        if node in chargers.keys():
            color_map.append('green')
        else: 
            color_map.append('blue')      

    # Draw Network
    labels = nx.get_edge_attributes(G,'weight')
    plt.figure(figsize=(11,9))
    nx.draw_networkx_edge_labels(G,pos, edge_labels=labels)
    nx.draw(G, pos, with_labels=True, alpha=0.5, node_color=color_map,node_size=1000)
    plt.show()
  except exception:
    print(exception)
    print("Objective Solution Not Found")

#### Network 1

In [ ]:
G1 = create_network(8953456, 8, 0.25)

#### Network 2

In [ ]:
G2 = create_network(45651451, 10, 0.25)

#### Network 3

In [ ]:
G3 = create_network(6546517, 12, 0.2)

# Implementation


## Creating Sets

In [ ]:
from typing_extensions import Self
import itertools


class GraphSets():
  """
  This Class is responsible for generating all the data related to a Graph.
  These attributes will be the Class' attributes that can be called
  """

  def __init__(self, G, NUMBER_OF_INTERSECTIONS):
    self.G = G
    self.NUMBER_OF_INTERSECTIONS = self.G.number_of_nodes()
    self.ARC_ROWS = set()
    self.ARC_MATRIX = []
    self.ROUTES = []
    self.routes = []
    self.ROUTE_DISTANCES = []
    self.ROUTE_ARCS = []
    self.ROUTE_ARC_BINARY = []
    # self.populate()

  def get_data(self):
    data = {}
    I = list(self.G.nodes) # intersections in the network
    A = self.ARC_MATRIX # lengths of feasible arcs/links between intersections
    R = self.ROUTES # intersections/nodes of routes
    L = self.ROUTE_DISTANCES # length of arcs/links of routes
    F = self.ROUTE_ARCS # arcs/links of routes
    data["I"] = I
    data["A"] = A
    data["R"] = R
    data["L"] = L
    data["F"] = F
    return data

  def populate(self):
    """
    This function will simply populate all the data for the Class attributes
    """
    
    graph = nx.to_dict_of_dicts(self.G)
    shortest_path = nx.dijkstra_path(self.G, 0, 2)

    shortest_paths = nx.all_pairs_dijkstra_path(self.G)

    arc_lengths = nx.get_edge_attributes(self.G,'weight')
    arc_lengths.update({(k[1], k[0]) : v for k,v in arc_lengths.items()})

    def get_arcs_from_nodes(node_list):
      return [(node_list[i-1], node_list[i]) for i in range (1, len(node_list))]

    
    for start_node in shortest_paths:
      for b in start_node[1].values():
        if len(b) > 1:
          arcs = get_arcs_from_nodes(b)
          arcs_w_lengths = {arc: arc_lengths[arc] for arc in arcs}
          arc_row = list(arcs_w_lengths.items())[0]
          self.ARC_ROWS.add(arc_row)
          self.routes.append({"nodes":tuple(b), "arcs":arcs_w_lengths})

    for i in range(self.NUMBER_OF_INTERSECTIONS):
      matrix_row = [0 for i in range(self.NUMBER_OF_INTERSECTIONS)]

      for row in self.ARC_ROWS:
        if row[0][0] == i:
          matrix_row[row[0][1]] = row[1]
      self.ARC_MATRIX.append(matrix_row)

    
    for j in self.routes:
      self.ROUTES.append((list(j["nodes"])))
      arc_lens = j['arcs']

      self.ROUTE_ARCS.append(list(arc_lens.keys()))

      arc_sum = sum(arc_lens.values())
      remaining_dist = []
      remaining_dist.append(arc_sum)
      for i in arc_lens.values():
        arc_sum -= i
        remaining_dist.append(arc_sum)
      self.ROUTE_DISTANCES.append(tuple(remaining_dist))


    matrix = np.zeros((len(self.ROUTES), self.NUMBER_OF_INTERSECTIONS, self.NUMBER_OF_INTERSECTIONS), dtype=int)

    for k in range(len(self.ROUTES)):
      arc = self.ROUTE_ARCS[k]

      for i in range(len(arc)):
        f = int(arc[i][0])
        s = int(arc[i][1])
        matrix[k][f][s] = 1

    self.ROUTE_ARC_BINARY = matrix
    return 

## Construction Heristic

In [ ]:
from collections import defaultdict

class ConstructionHeuristic():
  """ This class will create the Construction Heurisitic and return solution """
  RANGE_OF_CAR = 336

  def __init__(self, Data):
    self.I = Data["I"]
    self.A = Data["A"]
    self.R = Data["R"]
    self.L = Data["L"]
    self.F = Data["F"]
    self.sol_objective = None
    self.sol_charger_loc = None
    self.sol_charger_route = None
    self.total_route_length = 0

  def __objective_cost(self):
    # Charging station
    cost = 200_000 * len(self.sol_charger_loc) # Number of charging stations 
    # For each charging station:
    for c in self.sol_charger_loc.values():
      # Chargers at a charging station
      cost += 10_000 * c
    # Driving cost per km for all EVs
    cost += 0.02613 * self.total_route_length
    return cost

  def __solve(self):
    chargers_placed = defaultdict(lambda: 0)
    chargers_placed_route = defaultdict(set)
    for i, route in enumerate(self.F):
      remaining_charge = self.RANGE_OF_CAR
      for j, arc in enumerate(route):
        # Charge remaining after travelling to the next node

        arc_length = self.A[arc[0]][arc[1]]
        remaining_charge -= arc_length
        self.total_route_length += arc_length
        if remaining_charge < 0:
          # Since EV can not reach next node, add a charger at current node
          if chargers_placed[arc[0]] < 10:
            chargers_placed[arc[0]] += 1
            chargers_placed_route[arc[0]].add((route[0][0], route[-1][-1]))
          remaining_charge = self.RANGE_OF_CAR

    return [chargers_placed, chargers_placed_route]

  def get_solution(self):
    if self.sol_charger_loc is None:
      solved = self.__solve()
      self.sol_charger_loc = dict(solved[0])
      self.sol_charger_route = dict(solved[1])
    if self.sol_objective is None:
      self.sol_objective = self.__objective_cost()
    return (self.sol_objective, self.sol_charger_loc, self.sol_charger_route, self.total_route_length)


## Simulating Annealing (Meta-heuristic)

In [ ]:
from collections import defaultdict
from math import e
from random import random

class Simulated_Annealing():
  """ This class will create the Meta-Heurisitic and return solution """
  def __init__(self,G, Data):
    self.G = G
    self.I = Data["I"]
    self.A = Data["A"]
    self.R = Data["R"]
    self.L = Data["L"]
    self.F = Data["F"]

  def get_route_length(self, start_node, end_node):
     return nx.dijkstra_path_length(self.G,start_node, end_node)
  
  def get_route(self, start_node, end_node):
     return nx.dijkstra_path(self.G,start_node, end_node)

  def __objective_cost(self, chargers_placed, total_travel_distance):
    # Charging station
    cost = 200_000 * len(chargers_placed)
    for c in chargers_placed.values():
      # Chargers at a charging station
      cost += 10_000 * c
    # Driving cost per km for all EVs
    cost += 0.02613 * total_travel_distance
    return cost

  def will_acccept_candidate_solution(self, Z_n, Z_c, T):
    # For a minimization problem:
    if Z_n <= Z_c:
      # Accept since new solution is better then the current solution
      return True
    else:
      # Accept by probability
      P_accept = e**((Z_c-Z_n)/T)
      r = random()
      if r < P_accept:
        return True
      else:
        return False 

  def update_temperature(self, T, alpha):
    return alpha * T

  def get_new_feasible_route_length(self,start_node,end_node,charger,all_chargers):
    route_start = self.get_route(start_node,charger)
    route_end = self.get_route(end_node,charger)

    # Number of chargers passed on route
    ch_start = len(set(route_start).difference([start_node]).intersection(all_chargers))
    ch_end = len(set(route_end).difference([end_node]).intersection(all_chargers))

    dist_start = self.get_route_length(start_node,charger)
    dist_end = self.get_route_length(end_node,charger)

    if dist_start <= (RANGE_OF_CAR * (ch_start + 1)):
      # If EV can reach charger from start
      remaining_range =  (RANGE_OF_CAR * ch_start)-dist_start
      if dist_end <= (remaining_range + RANGE_OF_CAR * ch_start):
        # If EV can reach end from charger
        return dist_start + dist_end
      else:
        return None
    else:
      return None

  def find_alternate_routes(self, removed_charger, chargers, old_routes):
    # removed_charger = node from which chargers are to be remmoved
    # chargers = list of nodes with new chargers
    # old_routes = set of routes via old charger
    new_routes = defaultdict(set)
    change_in_distance = 0
    
    for route in old_routes:
      trip_length_via_charger = {}

      for node in chargers:
        # Get new route from start to end via this charger,
        # If this route is not feasible (i.e. EV will run out of charge), returns 'None'
        length = self.get_new_feasible_route_length(route[0],route[1],node,chargers)
        if length is not None:
          trip_length_via_charger[node] = length
      
      # Find the shortest route incorporating one of the new chargers.
      min_node = min(trip_length_via_charger, key=trip_length_via_charger.get)
      min_len = trip_length_via_charger[min_node]

      # old distance from start to end
      change_in_distance -= self.get_route_length(route[0],route[1])

      # new distance from start to end via new chargers
      change_in_distance += min_len

      new_routes[min_node].add((route[0],route[1]))
    
    return (new_routes, change_in_distance)
      

  def simulated_annealing(self, T_init, alpha, solution_c, Z_c, charger_routes, total_travel_distance_c, iterations):
    # Sort nodes in increasing number of chargers
    sorted_chargers = dict(sorted(solution_c.items(),key=lambda x: x[1]))
    # print("Solution at start of annealing:",
    #       f"Z_c = {Z_c}",
    #       f"Charger Placement = {sorted_chargers}",
    #       f"Routes in which EVs charge = {charger_routes}",
    #       f"Total distance travelled by EVs = {total_travel_distance_c}",
    #       f"Iterations left = {iterations}",
    #       f"Tempearature = {T_init}",
    #       f"Alpha = {alpha}",
    #       sep = "\n  ")
    
    T = T_init
    max_iterations_reached = False
    output = []

    # Find neighbor by swapping with lowest node from above list
    for node in sorted_chargers:
      skip = False
      for neighbor in self.G.neighbors(node):
        break_loop = False

        # Generate new list of possible chargers
        possible_charger_locations = set(sorted_chargers.keys()).difference([node]).union([neighbor])
        
        # Update routes
        # from the existing route list, remove the routes that need to be chaged and replace them with new routes. Same for distances.
        try:
          (routes_added,change_in_distance) = self.find_alternate_routes(node, possible_charger_locations, charger_routes[node])
          
          # Update charger routes and counts
          new_charger_routes = self.update_dict_remove_key_and_add_items(charger_routes.copy(), node, routes_added)
          # Create dict of new chargers
          chargers_added = { k: len(v) for k, v in routes_added.items()}
          new_solution = self.update_dict_remove_key_and_add_items(sorted_chargers.copy(), node, chargers_added)

        except ValueError:
          # If any of the new routes are infeasible, don't swap with this neighbor
          continue
          
        # the total route distance is used to calculate the cost of travelling on that route.
        total_travel_distance_n = total_travel_distance_c + change_in_distance

        Z_n = self.__objective_cost(new_solution, total_travel_distance_n)

        solution_accepted = False
        if self.will_acccept_candidate_solution(Z_n, Z_c, T):
          solution_accepted = True
          # Update current to neighbor
          Z_c = Z_n
          solution_c = new_solution
          sorted_chargers = dict(sorted(solution_c.items(),key=lambda x: x[1]))
          charger_routes = new_charger_routes
          total_travel_distance_c = total_travel_distance_n
          T = self.update_temperature(T,alpha)

          # Move to next node
          skip = True
          break_loop = True

        # Add neighbor solution to output
        output.append({"Z": Z_n, "chargers": new_solution, "accepted": solution_accepted})
        
        # Increment iterations
        iterations -= 1
        if iterations <= 0:
          max_iterations_reached = True
          break_loop = True

        # Test if we should break out of loop
        if break_loop:
          break
      
      if max_iterations_reached:
        break
      if skip:
        # This 'node' does not have a charger, so run simulated annealing with latest values
        output.extend(self.simulated_annealing(T,alpha,solution_c, Z_c, charger_routes, total_travel_distance_c, iterations))
        break

    # Return a collection of all feasible solutions
    return tuple(output)

  def update_dict_remove_key_and_add_items(self, dictionary, key_to_remove, items_to_add):
    # Remove key and its values
    del dictionary[key_to_remove]
    # Add new items
    for k, v in items_to_add.items():
      if k in dictionary.keys():
        # If key already exists, update existing data
        # Only update if existing value is an int or a set
        if isinstance(dictionary[k], set) and isinstance(v, set):
          dictionary[k] = dictionary[k].union(v)
        elif isinstance(dictionary[k], int) and isinstance(v, int):
          dictionary[k] += v
        else:
          raise TypeError(f"{type(dictionary[k])} and/or {type(v)} are not of a supported type of value. Supported types: (int, set)")
      else:
        # If no key exists, add data to new key
        dictionary[k] = v
    return dictionary

  def get_best_iteration(self, annealing_result):
    for i, result in reversed(list(enumerate(annealing_result))):
      if result["accepted"]:
        best = {}
        best['i'] = i
        best['Z'] = result['Z']
        best['chargers'] = result['chargers']
        return best

  def get_best_and_print_iterations(self, annealing_result):
    for i, result in enumerate(annealing_result):
      if result["accepted"]:
        result_string = f"Iteration: {i+1} | ACCEPTED |  Cost: {result['Z']}  |  Charger Placement: {result['chargers']}"
      else:
        result_string = f"Iteration: {i+1} | REJECTED |  Cost: {result['Z']}  |  Charger Placement: {result['chargers']}"
      print(result_string)
    best_result = self.get_best_iteration(annealing_result)
    print("================",
          "BEST SOLUTION:",
          f"Iteration: {best_result['i']} | Cost: {best_result['Z']}  |  Charger Placement: {best_result['chargers']}",
          sep="\n")
    return best_result


# Runs

## Base Run on Original Network

In [ ]:
Sets = GraphSets(G1["graph"], 8)
Sets.populate()
Data = Sets.get_data()

In [ ]:
# Construction Heuristic
Construction = ConstructionHeuristic(Data)
heuristic_solution1 = Construction.get_solution()

Z_c = heuristic_solution1[0]
placement_of_chargers = heuristic_solution1[1]
routes_of_chargers = heuristic_solution1[2]
total_travel_distance = heuristic_solution1[3]
print("Results of Construction Heuristic:",
      f"Z_c = {Z_c}",
      f"Charger Placement = {placement_of_chargers}",
      f"Routes in which EVs charge = {routes_of_chargers}",
      f"Total distance travelled by EVs = {total_travel_distance}",
      sep = "\n  ")

In [ ]:
# Simulated Annealing 
simulated_annealing_solution1 = Simulated_Annealing(G1["graph"],Data)
T_init = Z_c * 0.2
alpha = 0.5
result = simulated_annealing_solution1.simulated_annealing(T_init, alpha, placement_of_chargers, Z_c, routes_of_chargers, total_travel_distance, iterations=30)
metaheuristic_solution = simulated_annealing_solution1.get_best_and_print_iterations(result)

In [ ]:
print("Construction Heuristic")
display_solution(G1,heuristic_solution1[1], heuristic_solution1[0])
print("Simulated-Annealing Metaheuristic")
display_solution(G1,metaheuristic_solution["chargers"], metaheuristic_solution["Z"])

## Run on Network 2

In [ ]:
Sets2 = GraphSets(G2["graph"], 10)
Sets2.populate()
Data2 = Sets2.get_data()
print(Data2)

In [ ]:
# Construction Heuristic
Construction2 = ConstructionHeuristic(Data2)
heuristic_solution2 = Construction2.get_solution()

Z_c2 = heuristic_solution2[0]
placement_of_chargers2 = heuristic_solution2[1]
routes_of_chargers2 = heuristic_solution2[2]
total_travel_distance2 = heuristic_solution2[3]
print("Results of Construction Heuristic:",
      f"Z_c = {Z_c2}",
      f"Charger Placement = {placement_of_chargers2}",
      f"Routes in which EVs charge = {routes_of_chargers2}",
      f"Total distance travelled by EVs = {total_travel_distance2}",
      sep = "\n  ")

In [ ]:
# Simulated Annealing 
simulated_annealing_solution2 = Simulated_Annealing(G2["graph"],Data2)
T_init = Z_c2 * 0.2
alpha = 0.5
result2 = simulated_annealing_solution2.simulated_annealing(T_init, alpha, placement_of_chargers2, Z_c2, routes_of_chargers2, total_travel_distance2, iterations=30)
metaheuristic_solution2 = simulated_annealing_solution2.get_best_and_print_iterations(result2)

In [ ]:
print("Construction Heuristic")
display_solution(G2,heuristic_solution2[1], heuristic_solution2[0])
print("Simulated-Annealing Metaheuristic")
display_solution(G2,metaheuristic_solution2["chargers"], metaheuristic_solution2["Z"])

## Run on Network 3

In [ ]:
Sets3 = GraphSets(G3["graph"],12)
Sets3.populate()
Data3 = Sets3.get_data()

In [ ]:
# Construction Heuristic
Construction3 = ConstructionHeuristic(Data3)
heuristic_solution3 = Construction3.get_solution()

Z_c3 = heuristic_solution3[0]
placement_of_chargers3 = heuristic_solution3[1]
routes_of_chargers3 = heuristic_solution3[2]
total_travel_distance3 = heuristic_solution3[3]
print("Results of Construction Heuristic:",
      f"Z_c = {Z_c3}",
      f"Charger Placement = {placement_of_chargers3}",
      f"Routes in which EVs charge = {routes_of_chargers3}",
      f"Total distance travelled by EVs = {total_travel_distance3}",
      sep = "\n  ")

In [ ]:
# Simulated Annealing 
simulated_annealing_solution3 = Simulated_Annealing(G3["graph"],Data3)
T_init = Z_c3 * 0.2
alpha = 0.5
result3 = simulated_annealing_solution3.simulated_annealing(T_init, alpha, placement_of_chargers3, Z_c3, routes_of_chargers3, total_travel_distance3, iterations=30)
metaheuristic_solution3 = simulated_annealing_solution3.get_best_and_print_iterations(result3)

In [ ]:
print("Construction Heuristic")
display_solution(G3,heuristic_solution3[1], heuristic_solution3[0])
print("Simulated-Annealing Metaheuristic")
display_solution(G3,metaheuristic_solution3["chargers"], metaheuristic_solution3["Z"])

# Numerical Testing

Testing is essential for determining the validity and accuracy of a model/heuristic. Hence, we decided that we will generate three different random networks, and take the average of how each heuristic performs. Furthermore, we will also be using Alpha and Temperature values to test and find the optimal value. Alpha values will range from 0.05 to 1. Temperature parameters will start from 1 and be divided by 1.5 every iteration. The temperature will then be obtained by multiplying the temperature parameter by the Optimal Cost. 

In [ ]:
# Creating Alpha values that range from 0.05 -> 1
alpha_list = []
alphas = 0.05
for i in range(20):
  alpha_list.append(alphas)
  alphas += 0.05

In [ ]:
# Creating Temperature Params values that change arbitrarily from 1 -> near 0
temperature_params = []
temp = 1
for i in range(20):
  temperature_params.append(temp)
  temp /= 1.5

def temperature_values(Z_c):
  # Creating temperature values by multiplying the temperature params by Optimal Cost
  temperature = []
  for i in temperature_params:
    temperature.append(i*Z_c)
  return temperature  

In [ ]:
# Network 1
import sys
MAX_ITERATIONS = 30

# This wil be the test of all the feasible solutions and how they behave based on
# different temperature and alpha values
temp_list = temperature_values(heuristic_solution1[0])

# numerical_test[alpha][temp] = dict {"Z":int, "chargers": dict {node:num_chargers,...}}
numerical_test1 = defaultdict(lambda: defaultdict(dict))

# Simulated Annealing data
simulated_annealing_solution1 = Simulated_Annealing(G1["graph"],Data)

for alpha in alpha_list:
  for temp in temp_list:
    solution = simulated_annealing_solution1.simulated_annealing(temp, alpha, placement_of_chargers, Z_c, routes_of_chargers, total_travel_distance, MAX_ITERATIONS)
    best_solution = simulated_annealing_solution1.get_best_and_print_iterations(solution)
    numerical_test1[alpha][temp] = best_solution

min = sys.maxsize
optimal_locations = []
al = 0
temp = 0
Z_values = []

for i in alpha_list:
  for j in temp_list:
    Z_values.append(numerical_test1[i][j]["Z"])
    if numerical_test1[i][j]["Z"] < min:
      min = numerical_test1[i][j]["Z"]
      optimal_locations = numerical_test1[i][j]["chargers"]
      al = i
      temp = j


In [ ]:
print("The optimal location for Network 1 is ", optimal_locations, "at temperature", temp, "and alpha", al, "at cost", min)


In [ ]:
display_solution(G1,optimal_locations, min)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

X = alpha_list
Y = temp_list
Z = Z_values

x = np.reshape(X, (20))
y = np.reshape(Y, (20))
z = np.reshape(Z, (400))


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf1 = ax.plot_trisurf(x, y, z, cmap=cm.coolwarm)


ax.set_xlabel('Alpha')
ax.set_ylabel('Temperatures')
ax.set_zlabel('Z values')

plt.show()